In [ ]:
!python3 -m pip install tensorflow
!pip install transformers



In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from transformers import BertTokenizer, TFBertModel


# Load the data
train = pd.read_csv('/Users/dkamboj6/Recitation/ECE-461P-Term-Project/mercari-price-suggestion-challenge/train.tsv',  delimiter='\t')
test = pd.read_csv('/Users/dkamboj6/Recitation/ECE-461P-Term-Project/mercari-price-suggestion-challenge/test.tsv', delimiter='\t')

In [43]:
# Fill missing values
train
train.fillna('No Data', inplace=True)
test.fillna('No Data', inplace=True)


In [45]:
# Encode categorical variables using LabelEncoder
categorical_cols = ['category_name', 'brand_name']
encoder = LabelEncoder()

for col in categorical_cols:
    combined = pd.concat([train[col], test[col]], axis=0).astype(str)
    encoder.fit(combined)
    train[col] = encoder.transform(train[col].astype(str))
    test[col] = encoder.transform(test[col].astype(str))

# Scale numeric inputs
scaler = MinMaxScaler()
numeric_cols = ['item_condition_id', 'shipping']
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])
test[numeric_cols] = scaler.transform(test[numeric_cols])

# Log transform the price for normal distribution
train['log_price'] = np.log1p(train['price'])

In [48]:

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

def get_embeddings(text):
    # Check if text is None or empty and return a zero vector if true
    if not text or text == 'No Data':
        return np.zeros((768,))  # 768 is the dimensionality of BERT-base embeddings
    inputs = tokenizer(text, return_tensors="tf", max_length=128, truncation=True, padding="max_length")
    outputs = model(inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # Use the [CLS] token's embedding

# Apply the function to the DataFrame; consider doing this in batches if data is large
train['name_emb'] = train['name'].apply(lambda x: get_embeddings(x))
train['desc_emb'] = train['item_description'].apply(lambda x: get_embeddings(x))


TypeError: 'NoneType' object is not callable